In [24]:
import requests
import uuid
import time
from concurrent.futures import ThreadPoolExecutor, as_completed

In [25]:
STATE = '06'

COUNTIES = {"HUMBOLDT": "023",
            "LA": "037",
            "SF": "075",
            "SD": "073",
            "SACRAMENTO": "067"}

In [65]:
# date-range related
def generate_date_args(start_year=None, end_year=None):
    '''
    This just generates the correctly formatted date arg to pass.
    Years start on Jan 1
    :param start_year: YY (e.g. 2001 = 01)
    :param end_year: YY (max 21)
    :return: array of strings
    '''

     # We're actually only pulling to 2020 bc 2021 data isn't guaranteed yet

    if end_year:
        assert (end_year < 2021)
    else:
        end_year = 2021

    if start_year is None:
        start_year = 2014

    diff = range(start_year, end_year)
    all_years = list(diff)

    # EPA enforces limit of year inclusive
    start_year = "{YY}0101"
    end_year = "{YY}1231"

    all_date_tuples = []
    for YY in all_years:
        date_args = (start_year.format(YY=YY), end_year.format(YY=YY))
        all_date_tuples.append(date_args)

    return all_date_tuples



In [69]:
def make_urls(param=None, report_type=None):
    """
    dirty little hack for generating a list of urls for the thread
    pool executor to hit up
    :param param:
    :param report_type: str describing which report endpoint to hit, e.g Sample Data
    :return: list of strings
    """
    date_args = generate_date_args()

    if report_type is None:
        report_type = "annualData"

    # Defaulting to carbon emissions because that's the most common
    if param is None:
        param = "42101"

    base_url = "https://aqs.epa.gov/data/api/{report_type}" \
               "/byState?email=carrilloreb9@gmail.com&key=indigocat58".format(report_type=report_type)
    param_snippet = "&param={param}".format(param=param)

    all_urls = []
    
    for a, b in date_args:
        date_snippet = "&bdate={a}&edate={b}&state=06".format(a=a, b=b)
        full_url = base_url + param_snippet + date_snippet
        all_urls.append(full_url)

    return all_urls

In [70]:
urls = make_urls(param="42101", report_type="sampleData")

In [60]:
def download_file(url, file_name):
    
    try:
        data = requests.get(url)
        #open(f'(file_name).json', 'w').write(data)
        return data
    
    except e:
        return e
    
def runner():
    threads = []
    with ThreadPoolExecutor(max_workers=20) as executor:
        for url in urls:
            file_name = uuid.uuid1()
            threads.append(executor.submit(download_file, url, file_name))
        for task in as_completed(threads):
            print(task.result())
        

In [71]:
test_urls=urls[0:3]

In [72]:
test_urls

['https://aqs.epa.gov/data/api/sampleData/byState?email=carrilloreb9@gmail.com&key=indigocat58&param=42101&bdate=20140101&edate=20141231&state=06',
 'https://aqs.epa.gov/data/api/sampleData/byState?email=carrilloreb9@gmail.com&key=indigocat58&param=42101&bdate=20150101&edate=20151231&state=06',
 'https://aqs.epa.gov/data/api/sampleData/byState?email=carrilloreb9@gmail.com&key=indigocat58&param=42101&bdate=20160101&edate=20161231&state=06']

In [73]:
runner()

<Response [502]>
<Response [502]>


NameError: name 'e' is not defined